## 1. Setup `DataDog` agent 

- DataDog support different platform/machine (Linux, Docker, Cloud ...)
- We will use AWS EC2 instance to host `DataDog` agent
- Provision an Ubuntu EC2 instance in AWS

### Install `DataDog` agent in EC2

- Login into DataDog dashboard using signup account
- Follow setup `agent` documentation & select the host system as Ubuntu
- ssh to EC2 instance and run following command to setup agent

```bash
DD_AGENT_MAJOR_VERSION=7 DD_API_KEY=96e4c1f798656cba0491cca292cbc410 DD_SITE="datadoghq.com" bash -c "$(curl -L https://s3.amazonaws.com/dd-agent/scripts/install_script.sh)"
```

- Wait for the command run completely
- Run following command to check the status of `datadog-agent` service

```bash
sudo systemctl status datadog-agent 
```

- You should see following output:
```
● datadog-agent.service - Datadog Agent
     Loaded: loaded (/lib/systemd/system/datadog-agent.service; enabled; vendor preset: enabled)
     Active: active (running) since Sat 2020-11-21 15:17:14 UTC; 5s ago
   Main PID: 70805 (agent)
      Tasks: 7 (limit: 1164)
     Memory: 108.5M
     CGroup: /system.slice/datadog-agent.service
             └─70805 /opt/datadog-agent/bin/agent/agent run -p /opt/datadog-agent/run/agent.pid
```

- If you see following output. This means your agent has been stop. 
```
● datadog-agent.service - Datadog Agent
     Loaded: loaded (/lib/systemd/system/datadog-agent.service; enabled; vendor preset: enabled)
     Active: inactive (dead) since Sat 2020-11-21 10:58:34 UTC; 4h 16min ago
    Process: 31265 ExecStart=/opt/datadog-agent/bin/agent/agent run -p /opt/datadog-agent/run/agent.pid (code=exited, status=0/SUCCESS)
   Main PID: 31265 (code=exited, status=0/SUCCESS)
```

- You should start it by running command 
```bash
sudo systemctl start datadog-agent
```

- Now you should have your `datadog-agent` up and running in your host and start playing with its features 

### Feature 1: Logs - ingest a system log file to `DataDog` dashboard

#### Configuration

- 1. Collecting logs is disabled by default in the Datadog Agent. You must enable it in datadog.yaml

```bash
sudo vi /etc/datadog-agent/datadog.yaml

# enable collecting logs
logs_enabled: true
```

- 2. Define your `source` and `service`. For example: we could you `source=syslog` and `service=python`

```bash
sudo mkdir -p /etc/datadog-agent/conf.d/python.d/

# create conf.yaml file 
sudo vi /etc/datadog-agent/conf.d/python.d/conf.yaml

#Log section
logs:

    # - type : (mandatory) type of log input source (tcp / udp / file)
    #   port / path : (mandatory) Set port if type is tcp or udp. Set path if type is file
    #   service : (mandatory) name of the service owning the log
    #   source : (mandatory) attribute that defines which integration is sending the log
    #   sourcecategory : (optional) Multiple value attribute. Can be used to refine the source attribute
    #   tags: (optional) add tags to each log collected

  - type: file
    path: /var/log/syslog
    service: syslog
    source: python
```


- 3. Restart the agent to apply new configuration
```bash
sudo systemctl restart datadog-agent
```

#### Troubleshooting

```bash
# check status

sudo systemctl status datadog-agent


Nov 21 15:37:31 ip-172-31-42-32 agent[75632]: 2020-11-21 15:37:31 UTC | CORE | INFO | (pkg/logs/input/file/scanner.go:225 in startNewTailer) | Starting a new tailer for: /var/log/syslog (offset: 0, whence:>
Nov 21 15:37:31 ip-172-31-42-32 agent[75632]: 2020-11-21 15:37:31 UTC | CORE | INFO | (pkg/logs/input/file/tailer_nix.go:29 in setup) | Opening /var/log/syslog for tailer key /var/log/syslog
Nov 21 15:37:31 ip-172-31-42-32 agent[75632]: 2020-11-21 15:37:31 UTC | CORE | WARN | (pkg/logs/input/file/scanner.go:229 in startNewTailer) | open /var/log/syslog: permission denied
```

- The message said that `datadog-agent` doesn't have permission to tail the file `/var/log/syslog` since this is file under root
- So we should `chmod` the file to `644` so the agent could read it. However, this is not recommended way in term of security vulnerability
- We will copy this file to another user folder 

```bash
cp /var/log/syslog /home/ubuntu/01_setup_datadog/logs/

chmod 644 /home/ubuntu/01_setup_datadog/logs/syslog

# edit the conf.yaml file again to point to new file
#Log section
logs:

    # - type : (mandatory) type of log input source (tcp / udp / file)
    #   port / path : (mandatory) Set port if type is tcp or udp. Set path if type is file
    #   service : (mandatory) name of the service owning the log
    #   source : (mandatory) attribute that defines which integration is sending the log
    #   sourcecategory : (optional) Multiple value attribute. Can be used to refine the source attribute
    #   tags: (optional) add tags to each log collected

  - type: file
    path: /home/ubuntu/01_setup_datadog/logs/syslog
    service: syslog
    source: python
```

- Restart and check the status again

```bash
sudo systemctl restart datadog-agent

sudo systemctl status datadog-agent

Nov 21 15:47:51 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:51 UTC | CORE | INFO | (pkg/collector/runner/runner.go:327 in work) | check:disk | Done running check, next runs will be logged every 500 runs
Nov 21 15:47:56 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:56 UTC | CORE | INFO | (pkg/collector/runner/runner.go:261 in work) | check:network | Running check
Nov 21 15:47:56 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:56 UTC | CORE | INFO | (pkg/collector/runner/runner.go:327 in work) | check:network | Done running check, next runs will be logged every 500 r>
Nov 21 15:47:57 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:57 UTC | CORE | INFO | (pkg/collector/runner/runner.go:261 in work) | check:load | Running check
Nov 21 15:47:57 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:57 UTC | CORE | INFO | (pkg/collector/runner/runner.go:327 in work) | check:load | Done running check, next runs will be logged every 500 runs
Nov 21 15:47:58 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:58 UTC | CORE | INFO | (pkg/logs/input/file/scanner.go:248 in handleTailingModeChange) | Tailing mode changed for file:/home/ubuntu/01_setup_d>
Nov 21 15:47:58 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:58 UTC | CORE | INFO | (pkg/logs/input/file/scanner.go:225 in startNewTailer) | Starting a new tailer for: /home/ubuntu/01_setup_datadog/logs/>
Nov 21 15:47:58 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:58 UTC | CORE | INFO | (pkg/logs/input/file/tailer_nix.go:29 in setup) | Opening /home/ubuntu/01_setup_datadog/logs/syslog for tailer key /hom>
Nov 21 15:47:58 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:58 UTC | CORE | INFO | (pkg/collector/runner/runner.go:261 in work) | check:file_handle | Running check
Nov 21 15:47:58 ip-172-31-42-32 agent[77860]: 2020-11-21 15:47:58 UTC | CORE | INFO | (pkg/collector/runner/runner.go:327 in work) | check:file_handle | Done running check, next runs will be logged every 5>
lines 1-19/19 (END)
```

- Congrats. You did it successfully

### Feature 2. Infrastructure/Integration Metrics

- Login to `DataDog` portal
- Follow the documentation to create an integration with AWS which will all `datadog-agent` to pull our log/activity for your AWS to the portal
- https://docs.datadoghq.com/integrations/amazon_web_services/?tab=manual

- Here is basic idea of AWS integration
    - You will first need to create a IAM role in your AWS account which will allow `DataDog` aws account 
    
    ```
    {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "AWS": "arn:aws:iam::464622532012:root"
          },
          "Action": "sts:AssumeRole",
          "Condition": {
            "StringEquals": {
              "sts:ExternalId": "14df577b07ab47ac89934cdb9ec99704"
            }
          }
        }
      ]
    }
    ```
    
    - Create a policy to allow this role to access to your AWS services. You could also limit the access by specify the resource
    
    ```
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Action": [
                    "cloudwatch:Get*",
                    "cloudwatch:List*",
                    "ec2:Describe*",
                    "support:*",
                    "tag:GetResources",
                    "tag:GetTagKeys",
                    "tag:GetTagValues"
                ],
                "Effect": "Allow",
                "Resource": "*"
            }
        ]
    }
    ```
    
    - Install the integration in DataDog AWS integration tile.
    
    - Finally, you also need to restart your agent in the host to apply the new configuration

- By result, you could find all the data activity/log of your AWS in DataDog dashboard

### Feature 3. Alerting and Dashboarding

- Show detail in our sharing session. You could take screenshot by yourself

### Feature 4. Synthetics test

- This feature is kind of automation test for your API 

- You could create a sample API via API gateway or choose any simple public API for testing

- Here I will choose a public api: https://app.swaggerhub.com/apis/nekvapil/InternationalNamedayAPI/3.0.0

- Detail will be explain on our session